In [1]:
import pandas as pd
from gensim import corpora
from gensim import models

LDA模型就可以直接对电影的全部评论文本一次性做主题分析，但是如果把文本的正面评价（好评）和负面评价（差评）混淆在一起，并且由于分词粒度的影响（否定词或程度词等，可能在一个主题下生成一些令人迷惑的词语。因此抛弃中性的评论，对好评和差评分别进行LDA主题分析是一个比较好的主题。

In [2]:
#读取经过预处理的好评数据
pos = pd.read_csv('comments_sets/pre_pos.txt', encoding = 'utf-8', header = None)
#读取经过预处理的差评数据
neg = pd.read_csv('comments_sets/pre_neg.txt', encoding = 'utf-8', header = None)

In [3]:
pos.head()

,0
0,"['终局', '开局', '宇宙', '第一季', '结尾', '第二季', '引子', '..."
1,"['六个', '字']"
2,"['钢铁', '侠凉']"
3,"['还行', '还行']"
4,"['好看', '真的', '好看']"


In [4]:
neg.head()

,0
0,"['老套', '剧情', '开头', '结尾']"
1,['垃圾']
2,"['花', '钱', '娃去']"
3,"['对象', '黄了', '简单']"
4,"['感觉', '特技', '没什么', '可看']"


In [5]:
#定义一个分割函数，然后用 apply 广播
pos[1] = pos[0].apply(lambda s: s.split(' '))
neg[1] = neg[0].apply(lambda s: s.split(' '))

In [21]:
#去除标点符号
import string
for i in range(len(pos[1])):  # pos[1][i]: comments
    new_words = []
    for j in pos[1][i]:  # j: words
        trantab = str.maketrans({key: None for key in string.punctuation})
        new_word = j.translate(trantab)
        new_words.append(new_word)
    pos[1][i] = new_words

for i in range(len(neg[1])):  # pos[1][i]: comments
    new_words = []
    for j in neg[1][i]:  # j: words
        trantab = str.maketrans({key: None for key in string.punctuation})
        new_word = j.translate(trantab)
        new_words.append(new_word)
    neg[1][i] = new_words    

In [30]:
## 正面主题分析
pos_dict = corpora.Dictionary(pos[1])
pos_corpus = [pos_dict.doc2bow(i) for i in pos[1]]
pos_lda = models.LdaModel(pos_corpus, num_topics = 3, id2word = pos_dict)
print("\n正面评价")
for i in range(3):
	print("潜在主题%d : " %(i+1))
	print(pos_lda.print_topic(i) ) #输出每个主题


正面评价
潜在主题1 : 
0.056*"钢铁" + 0.051*"侠" + 0.021*"真的" + 0.021*"好看" + 0.020*"英雄" + 0.018*"十年" + 0.016*"死" + 0.015*"哭" + 0.013*"结局" + 0.013*"美队"
潜在主题2 : 
0.063*"好看" + 0.032*"爱" + 0.029*"遍" + 0.028*"三千" + 0.027*"喜欢" + 0.023*"超级" + 0.019*"一部" + 0.018*"I" + 0.013*"震撼" + 0.012*"特别"
潜在主题3 : 
0.035*"不错" + 0.031*"好看" + 0.028*"完美" + 0.016*"剧情" + 0.013*"感觉" + 0.012*"精彩" + 0.011*"小时" + 0.011*"年" + 0.011*"特效" + 0.010*"结局"


根据对有用特征词提取，

主题1中的高频特征词：钢铁侠，英雄，复联，美队，雷神，故事，太等

主题2中的高频特征词：十年，不错，好看，完美，结束，宇宙，完美，情怀，系列等

主题3中的高频特征词：好看，爱，三千遍，喜欢，英雄，中，爱，情怀，I，太等

综上对正面主题及其中高频特征词可以看出，复仇者联盟4受到好评的优点可以归结为：

1)钢铁侠，黑寡妇，美国队长，雷神等超级英雄，每一位都有自己的故事和背景，每个角色都独具个性的刻画。

2)对于“超级英雄片”的主题不忘初心，一晃十年，在这个急功近利的时代，他们的“匠心”显得弥足珍贵。

3)大概是说的里面的几条感情线剧情，越是长大越会被这种纯粹的东西所感动。

In [29]:
## 负面主题分析
neg_dict = corpora.Dictionary(neg[1])
neg_corpus = [neg_dict.doc2bow(i) for i in neg[1]]
neg_lda = models.LdaModel(neg_corpus, num_topics = 2, id2word = neg_dict)
print("\n负面评价")

for i in range(2):
	print("潜在主题%d : " %(i+1))
	print(neg_lda.print_topic(i) ) #输出每个主题


负面评价
潜在主题1 : 
0.034*"小时" + 0.032*"好看" + 0.022*"睡着" + 0.013*"想" + 0.011*"无聊" + 0.011*"难看" + 0.010*"睡觉" + 0.010*"剧情" + 0.009*"三个" + 0.009*"前"
潜在主题2 : 
0.016*"剧情" + 0.015*"太" + 0.013*"好看" + 0.011*"垃圾" + 0.010*"感觉" + 0.006*"时间" + 0.006*"情怀" + 0.006*"死" + 0.006*"真的" + 0.005*"英雄"


主题1的高频特征词：小时，不好看，睡着，无聊，人物，剧情等

主题2的高频特征词：剧情，英雄，太，垃圾，人物，时间，角色，穿越，bug等

相对而言，复仇者联盟4的不足之处（差评点）主要体现于：

1)三个小时剧情上安排拖拉，对小白可能就不是那么友好（不理解可能会成为催眠神器），有的热门人物到结局才出来露个面，有点遗憾。

2)剧情逻辑不严谨,就比如部分情节被认定违背了时间悖论。